In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
## Drop irrelevant columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [4]:
## Apply encoder techniques
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1


In [5]:
## OneHot encoder
onehot_encoder = OneHotEncoder()
geo_encoder = onehot_encoder.fit_transform(df[['Geography']])
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
onehot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder.get_feature_names_out(['Geography']))

In [8]:
## Onehot encoder columns add with original data
df = pd.concat([df.drop('Geography', axis=1), geo_encoded_df], axis=1)
df.head(3)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0


In [9]:
## Save the encoders and scaler

with open ('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open ('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder, file)

In [10]:
## Divide the dependent and indeppendent data

X = df.drop('Exited', axis=1)
y = df['Exited']

In [11]:
X.head(3)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0


In [12]:
## Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Scaler the train data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
## Save scaler file
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

ANN Implementation

In [14]:
## Import tensorflow library
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.models import load_model
import datetime

In [15]:
X_train.shape[1]

12

In [16]:
## Build ANN Model
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 connectedd with input layer
        Dense(32, activation='relu'), # Hl2
        Dense(1, activation='sigmoid') # Output layer
    ]
)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [19]:
## Compile the model
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [20]:
## Setup the tensorboard
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [21]:
## Setup the Earlytopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [22]:
## Train the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test),
    epochs = 100,
    callbacks = [tensorboard_callback, early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 0.3881 - accuracy: 0.8356 - val_loss: 0.3598 - val_accuracy: 0.8590
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3532 - accuracy: 0.8558 - val_loss: 0.3449 - val_accuracy: 0.8600
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3478 - accuracy: 0.8572 - val_loss: 0.3486 - val_accuracy: 0.8575
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3438 - accuracy: 0.8574 - val_loss: 0.3420 - val_accuracy: 0.8575
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3404 - accuracy: 0.8577 - val_loss: 0.3403 - val_accuracy: 0.8625
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3385 - accuracy: 0.8626 - val_loss: 0.3478 - val_accuracy: 0.8570
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3371 - accuracy: 0.8608 - val_loss: 0.3402 - val_accuracy: 0.85

In [23]:
## save the model as compatiable with keras
model.save('model.h5')

d:\Machine-Learning-Practicals\ANN-Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
## Load tensorboard extension
%reload_ext tensorboard

In [27]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6009 (pid 1696), started 23:27:02 ago. (Use '!kill 1696' to kill it.)